In [1]:
import numpy as np
import pandas as pd
import time

d = pd.read_csv("temp.csv")
d.head()

,PD1,GzmB,CD8a,CD103,CD56,CD25,CD4,CD19,CD49a,CD3,FoxP3
0,1.760079,2.241769,0.447316,0.951417,1.085694,0.365136,1.159030,0.744424,1.756490,1.208541,0.897291
1,2.504053,1.743894,0.725944,1.063522,1.146853,0.924040,2.429543,1.594169,3.008561,1.925828,0.740108
2,1.171469,1.729824,0.942350,0.670537,1.141923,0.644870,0.998817,0.800615,1.419161,1.141207,0.937181
3,1.464891,1.908043,0.499899,0.927789,0.677614,1.049692,0.864599,0.746302,1.478173,1.268048,0.886642
4,1.989203,1.786417,0.857825,0.784642,0.617290,1.169930,0.363880,0.945732,1.475951,1.183369,0.803843


In [2]:
import scanpy as sc
import anndata
import igraph
d = d.iloc[:,:11]
adata = anndata.AnnData(X=pd.DataFrame.to_numpy(d), obs=list(d.index.values), var=list(d.columns.values))
adata

AnnData object with n_obs × n_vars = 278186 × 11 
    obs: 0
    var: 0

In [6]:
print("Nearest neighbours...")
start = time.time()
sc.pp.neighbors(adata, n_neighbors=35)
print("DONE: {} s".format(time.time()-start))
print("Louvain clustering...")
mid = time.time()
sc.tl.louvain(adata, resolution=0.25)
print("DONE: {} s".format(time.time()-mid))
print("Number of clusters: ",max(np.array(adata.obs['louvain']).astype(int)+1))
d['cluster']=np.array(adata.obs['louvain']).astype(int)+1
d.to_csv("temp.csv", index=False)

Nearest neighbours...


C:\Users\adity\Anaconda3\lib\site-packages\numba\compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "..\..\..\..\Anaconda3\lib\site-packages\umap\nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


DONE: 307.738650560379 s
Louvain clustering...
DONE: 559.4269225597382 s
Number of clusters:  10
